**(1) IMPORTANDO BIBLIOTECAS**

In [444]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


np.random.seed(0)


**IMPORTANDO CSVs**

In [445]:
train_csv = pd.read_csv('../input/titanic/train.csv')
test_csv = pd.read_csv('../input/titanic/test.csv')

train_csv.describe()

**FAZENDO ANALISE SOBRE O BD**

In [446]:
train_csv.info()

**IDENTIFICAÇÃO DE VARIAVEIS**
- Não númericas -> Name, Sex, Ticket, Cabin, Embarked
- Numérica - Discretas -> Age, Pclass, SibSp, Parch, Fare, Survived
- Numérica - Continua -> PassengerId 

**IDENTIFICANDO NaN VALORES**

In [447]:
train_csv.isnull().sum()

train_new = train_csv.drop(columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'])
test_new = test_csv.drop(columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'])

#REMOVENDO AS LINHAS COM NaN

train_new = train_new.dropna()
test_new = test_new.fillna(0)

**(2) EXPLORANDO OS DADOS DO BD**

In [448]:
data = pd.DataFrame()
data = train_new.loc[:, ['Age', 'Survived']]
data = data.drop(data[data['Survived'] == 0].index).reset_index()
data = data.drop(columns=('index'))

def creat_coluns(datas, c_names, div):
    p = [0] * len(c_names)
    for x in c_names:
        p[c_names.index(x)] = datas['Age'].apply((lambda y: 1 if y > (int(x.split(':')[0])) and y <= (int(x.split(':')[1])) else None)).notna().sum()    
        p[c_names.index(x)] = (int(p[c_names.index(x)])/len(div))*100
    return p    

col_name = []
col_name = ['0:10', '10:20', '20:30' , '30:40', '40:50', '50:60', '60:70', '70:80']
train_age_bar = pd.DataFrame()
train_age_bar['Names'] = col_name
train_age_bar['Tot %'] = creat_coluns(data, col_name, data)

**PLOTANDO O GRÁFICO**

In [449]:
plt.figure(figsize=(30,25))
sns.set(font_scale = 5)

sns.barplot(x="Names", y="Tot %", data = train_age_bar)
plt.xlabel('Intervalo de Idades', fontsize = 50)
plt.ylabel('Quantidade de sobreviventes em porcentagem', fontsize = 50)
plt.title('RELAÇÃO INTERVALO DE IDADES X QUANTIDADE DE SOBREVIVENTES')

**CONSTRUINDO GRÁFICOS SOBRE AS VARIAVEIS DESCRITIVA**

In [450]:
dt = pd.DataFrame()
dt = train_new.loc[:,['Survived', 'Pclass']]
dt = dt.drop(dt[dt['Survived'] == 0].index).reset_index()
dt = dt.drop(columns=('index'))

pl = [0]*3
class_all = []
class_all = ['1', '2', '3']
data_new = pd.DataFrame()
data_new['Levels'] = class_all
for i in class_all:
    pl[class_all.index(i)]= dt['Pclass'].apply(lambda x: 1 if x == (class_all.index(i)+1) else 0).sum()
data_new['Sum'] = pl

**PLOTANDO O GRÁFICO**

In [451]:
plt.figure(figsize=(30,25))
sns.set(font_scale = 5)

sns.barplot(x="Levels", y="Sum", data = data_new)
plt.xlabel('Tipo De Classe', fontsize = 50)
plt.ylabel('Quantidade de sobreviventes', fontsize = 50)
plt.title('RELAÇÃO TIPO DE CLASSES X QUANTIDADE DE SOBREVIVENTES')

**(3) TREINAMENTO DE MODELO E SUBMISSÃO DAS PREDIÇÕES**

In [452]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [453]:
#CRIAR UMA FUNÇÃO
def change(train_csv):
    X = train_csv.drop(columns=['Name', 'Ticket', 'Cabin']).reset_index().drop(columns = ['index'])
    X['Embarked'] = X['Embarked'].apply(lambda x: 0 if x == 'C' else (1 if x == 'Q' else 2)).astype(int) #SEPARANDO O LOCAL DE EMBARQUE
    X['Fare'] = X['Fare'].fillna(0).astype(int)
    X['Age'] = X['Age'].fillna(0).astype(int)
    X['Sex'] = X['Sex'].apply(lambda x: 1 if x == 'female'else 0).astype(int)
    if ('Survived' in train_csv) == True:
        Y = X['Survived']
        X = X.drop(columns = ['Survived'])
        return X,Y
    return X
    


X, Y = change(train_csv)
X = X.drop(columns=['PassengerId'])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

In [454]:
train_csv.info()
test_csv.info()

****

**TREINANDO E VALIDANDO O MODELO**

In [455]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [456]:
model = RandomForestClassifier()
model.fit(X_train,Y_train)

predict = model.predict(X_test)

#VALIDANDO A AMOSTRA
score = accuracy_score(Y_test, predict)

print(score)
#COM UM PREDIÇÃO DE  82% NA MÉDIA É UMA PREDIÇÃO MUITO INTERESSANTE

**GERANDO A PREDIÇÃO E ENVIANDO ELA PARA AVALIAÇÃO**

In [460]:
#USANDO O CSV COMPLETO PRIMEIRO IREMOS FAZER A LIMPEZA DELE
X_test_CSV = change(test_csv)
Y_passangerId = X_test_CSV['PassengerId']
X_test_CSV = X_test_CSV.drop('PassengerId',axis=1)
model.fit(X_train, Y_train)

predict = model.predict(X_test_CSV)

#FAZENDO A JUNÇÃO
gender_submission = pd.DataFrame()
gender_submission['PassengerId'] = Y_passangerId
gender_submission['Survived'] = predict

gender_submission.to_csv('gender_submission.csv', index = False)